In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd
import json

In [2]:
def get_html(url):
    html = requests.get(url, headers={
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36"
    })

    html = BeautifulSoup(html.text, "html.parser")

    return html


In [5]:

def convert_string_to_float(price):
    return float(price.replace("R$ ", "").replace("%", "").replace(".", "").replace(",", "."))


def get_card_information(html_doc, card):
    return convert_string_to_float(html_doc.find(class_=card).find(
        class_="_card-body").find("span").get_text())


def get_investidor10_infos(fii):
    try:
        url = f"https://investidor10.com.br/fiis/{fii}"

        html = get_html(url)

        nome = html.find(
            class_='name-ticker').get_text().replace("\n", " ").strip()

        cotacao = get_card_information(html, "cotacao")
        dy = get_card_information(html, "dy")
        pvp = get_card_information(html, 'vp')

        indicadores = html.find(id='table-indicators').find_all(class_='cell')
        principais_indicadores = ['TIPO DE FUNDO',
                                  'SEGMENTO', 'ÚLTIMO RENDIMENTO', 'VACÂNCIA']
        indicadores_selecionados = {}
        for indicador in indicadores:
            if indicador.find(class_='desc').find('span').get_text().strip() in principais_indicadores:
                if indicador.find(class_='desc').find('span').get_text().strip() == "ÚLTIMO RENDIMENTO":
                    indicadores_selecionados[indicador.find(class_='desc').find('span').get_text(
                    ).strip()] = convert_string_to_float(indicador.find(class_='desc').find(class_='value').get_text().strip())
                else:
                    indicadores_selecionados[indicador.find(class_='desc').find('span').get_text(
                    ).strip()] = indicador.find(class_='desc').find(class_='value').get_text().strip()

        return {
            "nome": nome,
            "cotacao": cotacao,
            "dy": dy,
            "pvp": pvp,
            **indicadores_selecionados
        }
    except Exception as e:
        print(e)


def is_fiagro(fii):
    fiagros = ['VGIA11']
    if fii in fiagros:
        return True
    else:
        return False


def convert_to_float(string):
    try:
        return float(string.replace(".", "").replace(",", ".").replace("%", "").replace("R$ ", ""))
    except Exception as e:
        return string


def get_fii(fii):
    try:
        url = None
        if is_fiagro(fii):
            url = f"https://statusinvest.com.br/fiagros/{fii}"
        else:
            url = f"https://statusinvest.com.br/fundos-imobiliarios/{fii}"

        html = get_html(url)

        nome = html.find("h1").get_text()

        cotacao = convert_to_float(html.find(title="Valor atual do ativo").find(
            "strong").get_text())

        valorizacao_diaria = convert_to_float(html.find(
            title="Variação do valor do ativo com base no dia anterior").find("b").get_text())

        valorizacao_mensal = convert_to_float(html.find(
            title="Valorização no preço do ativo com base no mês atual").find("b").get_text())

        valorizacao_anual = convert_to_float(html.find(
            title="Valorização no preço do ativo com base nos últimos 12 meses").find("strong").get_text())

        dy = convert_to_float(html.find(
            title="Dividend Yield com base nos últimos 12 meses").find("strong").get_text())

        ultimos_12_dividendos = convert_to_float(html.find(
            title="Soma total de proventos distribuídos nos últimos 12 meses").find(class_="sub-value").get_text())

        pvp = convert_to_float(html.find_all(class_='top-info')[1].find_all(class_='info')[1].find(
            'strong').get_text())

        ultimo_rendimento = html.find(id="dy-info").find(class_="info")
        ultimo = {
            "ultimo_dividendo": convert_to_float(ultimo_rendimento.find('strong').get_text()),
            "ultimo_rendimento": convert_to_float(ultimo_rendimento.find_all('div')[1].find_all(class_='sub-info')[0].find('b').get_text()),
            "ultima_cotacao_base": convert_to_float(ultimo_rendimento.find_all('div')[1].find_all(class_='sub-info')[1].find('b').get_text()),
            "ultima_data_com": ultimo_rendimento.find_all('div')[1].find_all(class_='sub-info')[2].find('b').get_text(),
            "ultima_data_pagamento": ultimo_rendimento.find_all('div')[1].find_all(class_='sub-info')[3].find('b').get_text(),
        }

        proximo_rendimento = html.find(id="dy-info").find_next_sibling()
        proximo = {
            "proximo_dividendo": convert_to_float(proximo_rendimento.find(class_="info").find('strong').get_text()),
            "proximo_rendimento": convert_to_float(proximo_rendimento.find(class_="info").find_all('div')[1].find_all(class_='sub-info')[0].find('b').get_text()),
            "proxima_cotacao_base": convert_to_float(proximo_rendimento.find(class_="info").find_all('div')[1].find_all(class_='sub-info')[1].find('b').get_text()),
            "proxima_data_com": proximo_rendimento.find(class_="info").find_all('div')[1].find_all(class_='sub-info')[2].find('b').get_text(),
            "proxima_data_pagamento": proximo_rendimento.find(class_="info").find_all('div')[1].find_all(class_='sub-info')[3].find('b').get_text(),
        }

        investidor10_response = get_investidor10_infos(fii)

        return {
            "nome": nome.split('-')[0].strip(),
            "tipo": investidor10_response["TIPO DE FUNDO"],
            "segmento": investidor10_response["SEGMENTO"],
            "vacancia": convert_to_float(investidor10_response["VACÂNCIA"]),
            "cotacao": cotacao,
            "valorizacao_diaria": valorizacao_diaria,
            "valorizacao_mensal": valorizacao_mensal,
            "valorizacao_anual": valorizacao_anual,
            "dy": dy,
            "ultimos_12_dividendos": ultimos_12_dividendos,
            "pvp": pvp,
            **ultimo,
            **proximo,
        }
    except Exception as e:
        print(e)

In [6]:
infos = get_fii("VGIA11")

In [10]:
infos

{'nome': 'VGIA11',
 'tipo': 'Outro',
 'segmento': 'Fiagros',
 'vacancia': '-',
 'cotacao': 7.81,
 'valorizacao_diaria': 0.9,
 'valorizacao_mensal': -5.1,
 'valorizacao_anual': -17.27,
 'dy': 17.67,
 'ultimos_12_dividendos': 1.38,
 'pvp': 0.81,
 'ultimo_dividendo': 0.1,
 'ultimo_rendimento': 1.25,
 'ultima_cotacao_base': 8.02,
 'ultima_data_com': '13/05/2024',
 'ultima_data_pagamento': '20/05/2024',
 'proximo_dividendo': '-',
 'proximo_rendimento': '-',
 'proxima_cotacao_base': '-',
 'proxima_data_com': '-',
 'proxima_data_pagamento': '-'}

In [12]:
pd.DataFrame(data=[infos])

,nome,tipo,segmento,vacancia,cotacao,valorizacao_diaria,valorizacao_mensal,valorizacao_anual,dy,ultimos_12_dividendos,...,ultimo_dividendo,ultimo_rendimento,ultima_cotacao_base,ultima_data_com,ultima_data_pagamento,proximo_dividendo,proximo_rendimento,proxima_cotacao_base,proxima_data_com,proxima_data_pagamento
0,VGIA11,Outro,Fiagros,-,7.81,0.9,-5.1,-17.27,17.67,1.38,...,0.1,1.25,8.02,13/05/2024,20/05/2024,-,-,-,-,-


In [15]:
def get_brapi_infos(fii):
  url=f"https://brapi.dev/api/quote/{fii}?token=sznhQxMf3sHZ7XG4CVUQ7h"
  response = requests.get(url)
  return response.json()["results"][0]

In [16]:
get_brapi_infos("xpml11")

{'currency': 'BRL',
 'shortName': 'FII XP MALLSCI',
 'longName': 'Xp Malls Fundo Investimentos Imobiliarios',
 'regularMarketChange': 0.86,
 'regularMarketChangePercent': 0.745,
 'regularMarketTime': '2024-05-20T19:54:41.000Z',
 'regularMarketPrice': 116.24,
 'regularMarketDayHigh': 116.93,
 'regularMarketDayRange': '116.01 - 116.93',
 'regularMarketDayLow': 116.01,
 'regularMarketVolume': 89043,
 'regularMarketPreviousClose': 115.38,
 'regularMarketOpen': 116.63,
 'fiftyTwoWeekRange': '116.01 - 116.93',
 'fiftyTwoWeekLow': 116.01,
 'fiftyTwoWeekHigh': 116.93,
 'symbol': 'XPML11',
 'priceEarnings': None,
 'earningsPerShare': None,
 'logourl': 'https://s3-symbol-logo.tradingview.com/fii--big.svg'}

In [13]:
# Timestamp Unix em segundos
timestamp = 1716234899
dt = datetime.fromtimestamp(timestamp)
formatted_date = dt.strftime('%d/%m/%Y')
formatted_date

'20/05/2024'

In [7]:
with open("../data/fiis.json", "r", encoding="utf-8") as file:
    fiis = json.load(file)
fiis

[{'ativo': 'VGIA11', 'qtd': 70, 'category': 'Fiagro'},
 {'ativo': 'CPTS11', 'qtd': 60, 'category': 'Papel'},
 {'ativo': 'VISC11', 'qtd': 4, 'category': 'Shopping'},
 {'ativo': 'LVBI11', 'qtd': 4, 'category': 'Logístico'},
 {'ativo': 'XPLG11', 'qtd': 4, 'category': 'Logístico'},
 {'ativo': 'KNSC11', 'qtd': 45, 'category': 'Papel'},
 {'ativo': 'HSML11', 'qtd': 4, 'category': 'Shopping'},
 {'ativo': 'VGHF11', 'qtd': 40, 'category': 'Papel'},
 {'ativo': 'BCFF11', 'qtd': 40, 'category': 'FOF'},
 {'ativo': 'XPML11', 'qtd': 3, 'category': 'Shopping'},
 {'ativo': 'KNCR11', 'qtd': 3, 'category': 'Papel'},
 {'ativo': 'BTLG11', 'qtd': 3, 'category': 'Logístico'},
 {'ativo': 'VIUR11', 'qtd': 45, 'category': 'Híbrido'}]

In [4]:
pd.read_csv("../data/data.csv").

Index(['nome', 'tipo', 'segmento', 'vacancia', 'cotacao', 'valorizacao_diaria',
       'valorizacao_mensal', 'valorizacao_anual', 'dy',
       'ultimos_12_dividendos', 'pvp', 'ultimo_dividendo', 'ultimo_rendimento',
       'ultima_cotacao_base', 'ultima_data_com', 'ultima_data_pagamento',
       'proximo_dividendo', 'proximo_rendimento', 'proxima_cotacao_base',
       'proxima_data_com', 'proxima_data_pagamento'],
      dtype='object')